# 19. TF2 API 개요

## 1. 들어가며
지금까지 의식하지는 않았지만 딥러닝 실습을 위해서 마치 당연한 것처럼 Tensorflow V2 API를 활용해 왔다. 하지만 좋은 딥러닝 프레임워크에는 Tensorflow만 있는 것이 아니다. 아무래도 Tensorflow의 가장 강력한 경쟁자인 pyTorch를 언급하지 않을 수 없다. Tensorflow API와 달리 철저히 파이써닉하고 직관적인 API 설계와 쉬운 사용법으로 pyTorch는 지금도 수많은 사용자들의 지지를 받고 있다. 하지만 Tensorflow도 V2가 되면서 pyTorch가 가진 장점을 대부분 흡수하고, Keras라는 pyTorch와 아주 닮은 API를 Tensorflow의 표준 API로 삼으면서, Google이 가진 분산환경을 다루는 엄청난 기술력과 결합하여 더욱 강력한 딥러닝 프레임워크로 진화해왔다.

이번 시간에는 앞으로도 우리가 계속 사용하게 될 Tensorflow(TF) V2 API의 구성상의 개요를 파악하고, 보다 다양하고 깊이있게 Tensorflow를 이해하고 활용할 수 있는 기본기를 갖추고자 한다.

### 학습목표
---
* Tensorflow V2의 개요와 특징을 파악한다.
* Tensorflow V2의 3가지 주요 API 구성방식을 이해하고 활용할 수 있다.
* GradientTape를 활용해 보고 좀더 로우레벨의 딥러닝 구현방식을 이해한다.

## 2. Tensorflow2의 구성과 특징

### Tensorflow V1의 특징과 문제점
---
만약 텐서플로우같은 딥러닝 프레임워크가 없었다면, 딥러닝에 몸담은 엔지니어들의 업무 강도가 얼마나 더해지게 될까? 정말 단순한 Fully Connected Network 만을 구현하는 데도 꽤나 힘든 과정이었을 것이다. 그 중 가장 어려운 단계는 아무래도 딥러닝 모델의 그래디언트를 수식으로 구하고 그것을 바탕으로 backward propagation을 구현하는 것일 것이다. 만약 모델이 훨씬 복잡해진다면 그 복잡한 수식의 gradient를 구하기 위해 엄청나게 복잡한 미분식을 다루어야 할텐데 생각만 해도 아찔한 일일 것이다.

그러나, 일반적으로 우리는 딥러닝 모델을 구현하기 위해 그 모델의 gradient 수식을 계산하느라 골머리를 싸매지 않는다. pyTorch 등 다른 딥러닝 프레임워크들도 마찬가지이긴 하지만, Tensorflow는 forward propagation 방향의 모델만 설계하면 그 모델의 gradient를 사전에 미리 구해둘 수 있다. 이것을 가능하게 하기 위해 Tensorflow는 초기 V1때부터 독특한 설계 사상을 보유했는데, 그것은 바로 Tensorflow를 거대한 노드-엣지 사이의 `유향 비순환 그래프(Directed Acyclic Graph, DAG)`로 정의했다는 점이다. 그렇게 되면 노드와 노드를 연결하는 매 엣지마다 chain-rule을 기반으로 gradient가 역방향으로 전파될 수 있다는 간단하면서도 강력한 아이디어이다. 이런 방식을 Tensorflow의 `Graph Mode`라고 한다.

그러나, 이런 설계가 주는 큰 단점이 하나 있었다. Tensorflow V1은 딥러닝 모델을 구성하는 그래프를 그려나가는 부분과, 그 그래프 상에서 연산이 실제 진행되어과는 과정을 엄격하게 분리해 놓았다. 거기에 가장 중요한 것이 바로 `session`이라는 개념이었다. 그래서 그래프 사이에 벌어지는 모든 연산은 반드시 `session.run()` 안에서 수행되는 것으로 엄격히 제한되었다. 물론 이 방식이 주는 대규모 분산환경에서의 확장성과 생산성이라는 장점도 있었지만, Tensorflow V1은 기본적으로 사용하기가 어려웠다. 코드도 길고 지저분할 뿐더러, 파이써닉하지 않기 때문에 구현 방식 자체가 난이도가 높았다. 무엇보다도, 그래프를 다 만들어놓고 돌려봐야 비로소 모델 구성시의 문제가 드러나는데, 이렇게 문제가 발생했을 때 해결하기가 너무나 어렵고 복잡했기 때문이다.

### Eager Mode를 수용하여 Tensorflow V2가 되다.
---
그러나 PyTorch는 달랐다. PyTorch는 `Eager Mode`라는 것을 제안했는데, 이것은 딥러닝 그래프가 다 그려지지 않아도 얼마든지 부분 실행 및 오류검증이 가능했다. 그리고 코드도 간결하고 파이써닉한 설계를 가지고 있었기 때문에 개발자들에게 훨씬 쉽게 다가갈 수 있었다. 그래서 마침내 Tensorflow도 `Eager Mode`를 전격 수용하게 된다. 그리고 Keras라는 또하나의 매우 쉽고 간결한 머신러닝 프레임워크를 수용하여 Tensorflow V2의 표준 API 로 삼았다. 이런 변화가 Tensorflow의 사용방식을 어떻게 바꾸었는지 예시를 보면 알 수 있다.

```python
# 텐서플로 1.x
outputs = session.run(f(placeholder), feed_dict={placeholder: input})
# 텐서플로 2.0
outputs = f(input)
```

기본적으로 V2의 코드가 훨씬 직관적이고 간결하다. 하지만 여기엔 보이기에의 단순함 이상의 차이가 있다. `Session.run()`에 의존하지 않고, 그래프를 완성하지 않아도 부분적인 실행이 가능하다는 Eager Mode의 장점은 설계, 구현, 디버깅 전과정을 매우 쉽고 직관적으로 만들어준다.

그 외에도 Tensorflow V2가 이전 버전에 비해 가지는 다양한 장점들이 있다. 아래 글을 읽어보자.

[Overview of changes from TF V1 to TF V2](https://www.datasciencecentral.com/profiles/blogs/tensorflow-1-x-vs-2-x-summary-of-changes)

API 구성방식의 차이 이외에도, TF V2는 Tensorflow Hub를 통해 갖춘 데이터셋 등의 에코시스템, 다양한 분산환경 지원 등을 통해 더욱 쉽고도 강력한 기능을 갖추고 있다.

## 3. Tensorflow2 API로 모델 구성하기

### TensorFlow2 API 알아보기
---
그럼 가장 간단한 이미지 분류 문제를 풀어가면서 TensorFlow2(Tensorflow V2)를 활용한 모델링을 해보자.

오늘 소개할 TensorFlow2에서 딥러닝 모델을 작성하는 방법에는 크게 3가지가 존재하는데, 바로 `Sequential`, `Functional`, 그리고 `Model Subclassing`이다. `Sequential` 모델은 이미 몇번 사용해보았기 때문에 설명없이 넘어가겠다. `Functional`은 `Sequential`의 보다 일반화된 개념이다. 그리고 `Subclassing`은 클래스로 구현된 기존의 모델을 상속받아 자신만의 모델을 만들어나가는 방식이다. 세가지 방법 모두 충분히 숙지하여 자유롭게 활용할 수 있게 되는 것이 중요하다.

### 1) TensorFlow2 Sequential Model
---
```python
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential()
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)

model.fit(x, y, epochs=10, batch_size=32)
```

앞선 자료에서 공부했던 모델은 대부분 위와 같은 형식이었다. Sequential 모델을 활용하면 손쉽게 딥러닝 모델을 쌓아나갈 수 있다. 입력부터 출력까지 레이어를 그야말로 sequential하게 차곡차곡 add해서 쌓아나가기만 하면 된다. 무엇보다 이 방식은 초보자가 접근하기에 매우 쉽다는 장점이 있다. 그렇지만 모델의 입력과 출력이 여러 개인 경우에는 적합하지 않은 모델링 방식이다. Sequential 모델은 반드시 입력 1가지, 출력 1가지를 전제로 한다.

아래 참고자료에서 Sequential Model로 작성한 모델의 전체 코드를 확인해보자.

* [텐서플로 2.0 시작하기: 초보자용](https://www.tensorflow.org/tutorials/quickstart/beginner)

### 2) TensorFlow2 Functional API
---
```python
import tensorflow as tf
from tensorflow import keras

inputs = keras.Input(shape=(__원하는 입력값 모양__))
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(input)
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)
outputs = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.fit(x,y, epochs=10, batch_size=32)
```

여기서 위 `Sequential Model`을 활용하는 것과 다른 점은 바로 `keras.Model`을 사용한다는 점이다. 그래서 `Sequential Model`을 쓰는 것보다 더 일반적인 접근인 것이다. `Sequential Model`이란 사실 `keras.Model`을 상속받아 확장한 특수사례에 불과한 것이기 때문이다. `Functional API`를 활용하면 앞서 배운 `Sequential Model`을 활용하는 것보다 더 자유로운 모델링을 진행할 수 있다. <br>
함수형으로 모델을 구성한다는 것, 즉 __입력과 출력을 규정함으로써 모델 전체를 규정__한다는 생각이다. 그래서 이번에는 `Input`이라는 것을 규정한다. `Input`이 될 수 있는 텐서가 여러 개가 될 수 도 있다. 그리고 레이어들을 자유롭게 엮어 출력(Output)까지 규정하면 Model이란 바로 `inputs`와 `outputs` 만으로 규정된다. 정말 Functional하다.

Sequential Model의 제약점이 1개의 입력/출력이였다면 Functional API를 통해 다중입력/출력을 가지는 모델을 구성할 수 있다.

아래 참고자료에서 Functional API를 활용해서 모델링하는 방법에 대해서 보자.

* [The Keras functional API](https://www.tensorflow.org/guide/keras/functional)

### 3) TensorFlow2 Subclassing
---
```python
import tensorflow as tf
from tensorflow import keras

class CustomModel(keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()

    def call(self, x):
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)

        return x

model = CustomModel()
model.fit(x,y, epochs=10, batch_size=32)
```

마지막으로 `Subclassing`을 활용하면 제일 자유로운 모델링을 진행할 수 있다. 사실 본질적으로는 Functional한 접근과 차이가 없다. 이것은 `keras.Model`을 상속받은 모델 클래스를 만드는 것이기 때문이다. 처음 만났던 `Sequential Model`도 따지고 보면 `keras.Model`을 상속받은 모델 클래스의 하나일 뿐이다. `keras.Model`은 위와 같이 `__init__()`이라는 메소드 안에서 레이어 구성을 정의한다. 그리고 `call()`이라는 메소드 안에서 레이어간 `forward propagation`을 구현한다. 이것으로 끝이다. 다만, 각 레이어에 대한 깊은 이해가 필요하고 초심자에게 의도치 않은 버그를 유발할 수 있다. 그렇지만 복잡한 모델링을 진행하게 되면 가장 많이 접할 모델링 스타일이기에 모델링을 해볼 예정이다.

아래 참고자료에서 Subclassing을 활용한 모델링에 대해서 보자.

* [텐서플로 2.0 시작하기: 전문가용](https://www.tensorflow.org/tutorials/quickstart/advanced)

## 4. Tensorflow2 API로 모델 작성하기: MNIST (1) Sequential API 활용

앞에서 다층 신경망과 Convolutional Neural Network의 차이를 공부했고, TensorFlow2의 다양한 High-level API에 대해 둘러보았다. 지금은 앞서 본 TensorFlow2의 다양한 High-level API를 활용해서 이미지 문제를 풀어볼 예정이다.

총 2가지의 문제를 3가지 API 모두를 활용하여 구현해볼 예정이다.

잘 알려진 MNIST 문제를 구현해보도록 하자.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [4]:
model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# Spec:
# 1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
# 2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
# 3. Flatten 레이어
# 4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
# 5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)

In [5]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1092 - accuracy: 0.9670
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0332 - accuracy: 0.9898
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0194 - accuracy: 0.9937
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0127 - accuracy: 0.9957
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0087 - accuracy: 0.9972
313/313 - 3s - loss: 0.0512 - accuracy: 0.9872


[0.05121546611189842, 0.9872000217437744]

## 5. Tensorflow2 API로 모델 작성하기: MNIST (2) Functional API 활용
이번에는 `keras.Model`을 직접 활용하여야 하므로, `keras.Input`으로 정의된 input및 output 레이어 구성을 통해 model을 구현하여야 한다.

In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [7]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [8]:
inputs = keras.Input(shape=(28, 28, 1))

x = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
x = keras.layers.Conv2D(64, 3, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)


# Spec:
# 0. (28X28X1) 차원으로 정의된 Input
# 1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
# 2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
# 3. Flatten 레이어
# 4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
# 5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)


In [9]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1053 - accuracy: 0.9678
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0354 - accuracy: 0.9892
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0206 - accuracy: 0.9931
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0133 - accuracy: 0.9957
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0080 - accuracy: 0.9974
313/313 - 0s - loss: 0.0450 - accuracy: 0.9884


[0.045025646686553955, 0.9883999824523926]

## 6. Tensorflow2 API로 모델 작성하기: MNIST (3) Subclassing 활용
마지막으로 `Subclassing` 방법이다. `keras.Model`을 상속받은 클래스를 만드는 것이다. `__init__()` 메소드 안에서 레이어를 선언하고, `call()` 메소드 안에서 `forward propagation`을 구현하는 방식이다. `Functional` 방식과 비교하자면, `call()`의 입력이 Input이고, `call()`의 리턴값이 Output이 되는 것이다.

In [10]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [11]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [12]:
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = keras.layers.Conv2D(64, 3, activation='relu')
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(128, activation='relu')
        self.fc2 = keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

# Spec:
# 0. keras.Model 을 상속받았으며, __init__()와 call() 메소드를 가진 모델 클래스
# 1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
# 2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
# 3. Flatten 레이어
# 4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
# 5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
# 6. call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output

In [13]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1066 - accuracy: 0.9670
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0321 - accuracy: 0.9901
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0183 - accuracy: 0.9939
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0128 - accuracy: 0.9960
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0092 - accuracy: 0.9971
313/313 - 0s - loss: 0.0468 - accuracy: 0.9885


[0.0468430370092392, 0.9884999990463257]

## 7. TensorFlow2 API로 모델 작성 및 학습하기: CIFAR-100 (1) Sequential API 활용

먼저, 아래 링크를 방문해 CIFAR-100 문제가 무엇인지 확인해보자.

* [CIFAR-100 dataset](https://www.cs.toronto.edu/~kriz/cifar.html)

In [14]:
import tensorflow as tf
from tensorflow import keras

In [15]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [16]:
model = keras.Sequential([
    keras.layers.Conv2D(16, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(100, activation='softmax')
])


# Spec:
# 1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
# 2. pool_size가 2인 MaxPool 레이어
# 3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
# 4. pool_size가 2인 MaxPool 레이어
# 5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
# 6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)

In [17]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 11s 7ms/step - loss: 3.6152 - accuracy: 0.1526
Epoch 2/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.9043 - accuracy: 0.2832
Epoch 3/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.5737 - accuracy: 0.3511
Epoch 4/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.3578 - accuracy: 0.3946
Epoch 5/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.1837 - accuracy: 0.4357
313/313 - 0s - loss: 2.5887 - accuracy: 0.3546


[2.5886874198913574, 0.3546000123023987]

## 8. Tensorflow2 API로 모델 작성 및 학습하기: CIFAR-100 (2) Functional API 활용
이번에는 keras.Model을 직접 활용하여야 하므로, keras.Input으로 정의된 input및 output 레이어 구성을 통해 model을 구현해야 한다.

In [18]:
import tensorflow as tf
from tensorflow import keras

In [19]:
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [20]:
inputs = keras.Input(shape=(32, 32, 3))

x = keras.layers.Conv2D(16, 3, activation='relu')(inputs)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

# Spec:
# 0. (32X32X3) 차원으로 정의된 Input
# 1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
# 2. pool_size가 2인 MaxPool 레이어
# 3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
# 4. pool_size가 2인 MaxPool 레이어
# 5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
# 6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)


In [21]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 3s 2ms/step - loss: 3.6016 - accuracy: 0.1615
Epoch 2/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.8594 - accuracy: 0.2932
Epoch 3/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.5680 - accuracy: 0.3530
Epoch 4/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.3654 - accuracy: 0.3954
Epoch 5/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.1955 - accuracy: 0.4305
313/313 - 0s - loss: 2.6019 - accuracy: 0.3556


[2.6018974781036377, 0.3555999994277954]

## 9. Tensorflow2 API로 모델 작성 및 학습하기: CIFAR-100 (3) Subclassing 활용
마지막으로 `Subclassing` 방법이다. `keras.Model`을 상속받은 클래스를 만드는 것이다. `__init()__` 메소드 안에서 레이어를 선언하고, `call()` 메소드 안에서 `forward propagation`을 구현하는 방식임을 기억하자. `Functional` 방식과 비교하자면, `call()`의 입력이 Input이고, `call()`의 리턴값이 Output이 되는 것이다.

In [22]:
import tensorflow as tf
from tensorflow import keras

In [23]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [24]:
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

# Spec:
# 0. keras.Model 을 상속받았으며, __init__()와 call() 메소드를 가진 모델 클래스
# 1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
# 2. pool_size가 2인 MaxPool 레이어
# 3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
# 4. pool_size가 2인 MaxPool 레이어
# 5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
# 6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
# 7. call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output

In [25]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 3s 2ms/step - loss: 3.6044 - accuracy: 0.1590
Epoch 2/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.9073 - accuracy: 0.2828
Epoch 3/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.5906 - accuracy: 0.3474
Epoch 4/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.3691 - accuracy: 0.3950
Epoch 5/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.2007 - accuracy: 0.4310
313/313 - 0s - loss: 2.5911 - accuracy: 0.3539


[2.5911459922790527, 0.3538999855518341]

## 10. GradientTape의 활용

### Automatic differentiation - GradientTape
---
조금 전까지 아주 비슷한 테스크 2개를, 본질적으로 큰 차이가 없는 3개의 모델 구성 방법을 활용하여 딥러닝으로 구현해 보았다. 그동안 완전히 동일하게 구성했던 것은 바로 아래와 같이 구성된 모델 학습 관련 부분이다.

```python
# 모델 학습 설정
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
```

Numpy만 가지고 딥러닝을 구현하는 것을 회상해보자. `model.fit()`이라는 한줄로 수행 가능한 딥러닝 모델 훈련 과정은 실제로는 어떠했나?

* Forward Propagation 수행 및 중간 레이어값 저장
* Loss 값 계산
* 중간 레이어값 및 Loss를 활용한 체인룰(chain rule) 방식의 역전파(Backward Propagation) 수행
* 학습 파라미터 업데이트

이상 4단계로 이루어진 train_step 을 여러번 반복했다.

이런 과정이 TF2 API에는 `model.fit()`이라는 메소드 안에 모두 추상화되어 감추어져 있다.

Tensorflow에서 제공하는 `tf.GradientTape`는 위와 같이 순전파(forward pass) 로 진행된 모든 연산의 중간 레이어값을 `tape`에 기록하고, 이를 이용해 gradient를 계산한 후 `tape`를 폐기하는 기능을 수행한다. 아래에서 소개할 `tf.GradientTape`는 이후 그래디언트를 좀더 고급스럽게 활용하는 다양한 기법을 통해 자주 만나게 될 것이다.

In [26]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

# 모델 구성부분
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

50000 10000


여기까지는 앞에서 다루었던 Subclassing을 활용한 모델 작성법과 전혀 다르지 않다. 달라지는 것은 `model.compile()`, `model.fit()`을 통해 손쉽게 진행했던 학습 세팅 및 수행 부분이다.

```python
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
```

위와 같이 모델 학습을 위해 loss, optimizer를 지정해 주면 내부적으로는 매 스텝 학습이 진행될 때마다 발생하는 loss 및 그래디언트가 어떻게 학습 파라미터를 업데이트하게 되는지를 지정해 주는 작업이 `model.compile()` 안에서 자동으로 진행되었다. 아래 코드는 `tape.gradient()`를 통해 매 스텝 학습이 진행될 때마다 발생하는 그래디언트를 추출한 후 `optimizer.apply_gradients()`를 통해 발생한 그래디언트가 업데이트해야 할 파라미터 `model.trainable_variables`를 지정해 주는 과정을 기술한 것이다.

In [27]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

위와 같이 매 스텝 진행되는 학습의 실제 동작이 `train_step()` 메소드로 구현되었다.

```python
model.fit(x_train, y_train, epochs=5, batch_size=32)
```

이 `model.fit()`으로 위와 같이 한줄로 간단하게 수행되던 실제 배치 학습 과정은, 다름 아니라 매 스텝마다 위에서 구현했던 `train_step()`가 호출되는 과정으로 바꾸어 구현할 수 있다. `model.fit()` 호출시에 결정되는 `batch_size`만 이번 스텝에서 결정해 주면 된다.

In [28]:
import time
def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            if step % batch_size == batch_size-1:
                x_batch.append(x)
                y_batch.append(y)
                loss = train_step(np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.float32))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))

train_model()

Epoch 0: last batch loss = 4.5189
Epoch 1: last batch loss = 4.5334
Epoch 2: last batch loss = 4.5545
Epoch 3: last batch loss = 4.4561
Epoch 4: last batch loss = 4.9434
It took 78.81462359428406 seconds


위에서 구현한 `train_model()` 메소드가 실은 그동안 사용했던 `model.fit()` 메소드와 기능적으로 같다는 것을 확인 할 수 있다.

이렇듯 `tf.GradientTape()`를 활용하면 `model.compile()`과 `model.fit()` 안에 감추어져 있던 한 스텝의 학습 단계(위 예제에서는 `train_step` 메소드)를 끄집어내서 자유롭게 재구성할 수 있게 된다. 그동안 흔히 다루어 왔던 지도학습 방식과 다른 강화학습 또는 GAN(Generative Advasarial Network)의 학습을 위해서는 `train_step` 메소드의 재구성이 필수적이므로 `tf.GradientTape()`의 활용법을 꼭 숙지해 두어야 한다.

In [29]:
# evaluation
prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test)  # Accuracy

1/1 [==============================] - 0s 1ms/step


0.0589

그래디언트를 활용할 필요가 없는 evaluation 단계는 기존 `model.predict()` 메소드를 다시 활용하여 보았다. 충분한 성능을 확인할 수 있을 만큼의 학습이 진행된 상태가 아니니 최종 Accuracy 값은 신경쓰지 않아도 무방하다.